In [11]:
# importing necessary libraries
import numpy as np
import cv2
#from matplotlib import pyplot as plt
import os
import imutils
import math
import csv
import pandas as pd

path = './raw_in/unified'
fields = ['file_name', 'angle_of_crack', 'class']
df = pd.DataFrame(columns=fields)

with os.scandir(path) as it:
    for entry in it:
        if entry.name.endswith(".jpg") and entry.is_file():
            img = cv2.imread(entry.path, cv2.IMREAD_GRAYSCALE)
            original_img = cv2.imread(entry.path, cv2.IMREAD_COLOR)
            blur = cv2.GaussianBlur(img, (5, 5), 0)
            canny = cv2.Canny(blur, 50, 250)

            # Morphological Closing Operator
            kernel = np.ones((5, 5), np.uint8)
            closing = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)

            try:
                cnts = cv2.findContours(
                    closing, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

                cnts = cnts[0] if len(cnts) == 2 else cnts[1]
                c = max(cnts, key=cv2.contourArea)
                
                extLeft = tuple(c[c[:, :, 0].argmin()][0])
                extRight = tuple(c[c[:, :, 0].argmax()][0])
                extTop = tuple(c[c[:, :, 1].argmin()][0])
                extBot = tuple(c[c[:, :, 1].argmax()][0])

                cv2.drawContours(original_img, [c], -1, (0, 255, 255), 2)
                cv2.circle(original_img, extLeft, 4, (0, 0, 255), -1)
                cv2.circle(original_img, extRight, 4, (0, 255, 0), -1)
                cv2.circle(original_img, extTop, 4, (255, 0, 0), -1)
                cv2.circle(original_img, extBot, 4, (255, 255, 0), -1)

                cv2.line(original_img, extTop, extBot, (255,255,0), 5)

                m1 = (extBot[1] - extTop[1]) / (extBot[0] - extTop[0])
                
                angle = math.degrees(math.atan(m1))
                
                crack_class = None
                if (angle < 0):
                    if (abs(angle) > 45):
                        new_path = './processed_in/verticle_right/'
                        crack_class = 'verticle_right'
                    else:
                        new_path = './processed_in/horizonal_right/'
                        crack_class = "horizonal_right"
                else:
                    if(abs(angle) > 45):
                        new_path = './processed_in/verticle_left/'
                        crack_class = "verticle_left"
                    else:
                        new_path = './processed_in/horizontal_left/'
                        crack_class = "horizontal_left"
                
                print (entry.name,",", angle, ",", crack_class)
                df.append({'file_name': entry.name, 'angle_of_crack': angle, 'class': crack_class}, ignore_index=True)
                cv2.imwrite(new_path +
                            entry.name, original_img)
            except Exception as e:
                print(e)

            #plt.subplot(121), plt.imshow(img)
            #plt.title('Original'), plt.xticks([]), plt.yticks([])
            #plt.subplot(122), plt.imshow(featuredImg, cmap='gray')
            #plt.title('Output Image'), plt.xticks([]), plt.yticks([])
            #plt.show()
df.to_csv('./processed_in/result.csv', sep=',', encoding='utf-8')


07690.jpg , -45.45835645800043 , verticle_right
07143.jpg , 57.5876933816488 , verticle_left
06318.jpg , 46.12330271407543 , verticle_left
14223_1.jpg , -34.99202019855866 , horizonal_right
02121.jpg , -58.73626830562258 , verticle_right
06176.jpg , 61.64503821467594 , verticle_left
09241.jpg , 49.398705354995535 , verticle_left
11156_1.jpg , -65.2825590889166 , verticle_right
11631_1.jpg , 66.80140948635182 , verticle_left
07831.jpg , 20.973493421320807 , horizontal_left
03888.jpg , 23.051300916473025 , horizontal_left
13130_1.jpg , -52.1250163489018 , verticle_right
11245_1.jpg , -54.46232220802562 , verticle_right
04317.jpg , 57.9946167919165 , verticle_left
00627.jpg , 42.66269414087618 , horizontal_left
13839_1.jpg , 77.47119229084849 , verticle_left
18171_1.jpg , -28.024500532070256 , horizonal_right
12085_1.jpg , -79.2157021324374 , verticle_right
13773_1.jpg , 69.94390526342458 , verticle_left
06888.jpg , 38.418055344821994 , horizontal_left
17433_1.jpg , -61.64503821467594 , v

/tmp/ipykernel_127081/1707216457.py:47: RuntimeWarning: divide by zero encountered in int_scalars
  m1 = (extBot[1] - extTop[1]) / (extBot[0] - extTop[0])


12553_1.jpg , 59.57421619803874 , verticle_left
00232.jpg , 31.23921488820876 , horizontal_left
11659_1.jpg , 75.74996730219641 , verticle_left
03641.jpg , -38.29016319224307 , horizonal_right
10582_1.jpg , 63.799885158652664 , verticle_left
07730.jpg , -38.32011393677745 , horizonal_right
11829_1.jpg , 59.743562836470744 , verticle_left
14112_1.jpg , -53.8646233884108 , verticle_right
04712.jpg , -51.00900595749453 , verticle_right
16778_1.jpg , -69.94390526342458 , verticle_right
03076.jpg , 78.340707346477 , verticle_left
00665.jpg , 39.351752626264734 , horizontal_left
03470.jpg , 64.23067237566129 , verticle_left
15076_1.jpg , -38.32011393677745 , horizonal_right
02720.jpg , 46.24536426676835 , verticle_left
01709.jpg , 39.053136946026505 , horizontal_left
02364.jpg , -83.6598082540901 , verticle_right
12303_1.jpg , -35.36246188706905 , horizonal_right
03539.jpg , 38.927543592792304 , horizontal_left
16370_1.jpg , -22.83365417791754 , horizonal_right
07254.jpg , -27.32389130134654